## Import Libraries

In [1]:
import os
import time
import google.generativeai as genai
import ast
import re
import pandas as pd
from tqdm import tqdm
import time

## Configuration

In [2]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]

In [3]:
genai.configure(api_key="API-KEY") # Add your API_KEY not mine

model = genai.GenerativeModel('gemini-1.5-flash', safety_settings=safety_settings)

## Read the Data

In [4]:
anxiety_total_data = pd.read_excel("../Data/Expurgated_Data/Reddit/pmdd_expurgated.xlsx") # You can change it
anxiety_total_data

,Result,Text,Translated_Text
0,yes,"I'm going crazy, I'm in the first healthy rela...","Deliriyorum, hayatımın ilk sağlıklı ilişkisind..."
1,yes,I feel my crazy emotions returning.,Çılgın duygularımın geri döndüğünü hissediyorum.
2,yes,I'm so tired of going from good to bad...I'm j...,İyiden kötüye gitmekten çok yoruldum...Aşağılı...
3,yes,I constantly feel like i’m on the verge of ful...,Sürekli olarak kendimi tamamen kaybetmenin ve ...
4,yes,I’m really stuck in my life right now and need...,Şu anda gerçekten hayatımda sıkışıp kaldım ve ...
...,...,...,...
1495,yes,"I thought it was just plain ol’ PPD, but now t...","Bunun sadece düz bir PPD olduğunu düşündüm, an..."
1496,yes,"I have panicked and went to the doctor, got an...","Panikledim ve doktora gittim, EKG aldım. İlk b..."
1497,yes,I hate my body and self,Bedenimden ve kendimden nefret ediyorum.
1498,yes,"I've had bad cramps, a heavy flow and just don...","Kötü kramplar, ağır bir akış yaşadım ve sadece..."


## Augmentation (Main process)

In [5]:
mistake_sentences = []

In [6]:
def submission_prediction(sentence: str):
    global mistake_sentences
    
    response = model.generate_content("""
    Sen verilen cümleyi genel anlam ve kelime anlamı olarak analiz edip bu cümleden esinlenerek benzer cümleler oluşturan bir sentetik veri üreticisisin.

    \n\n

    Aşağıda verilen cümleyi genel anlam ve kelime anlamı olarak analiz et ve anla. Anlamı anladıktan sonra bu cümleyi söyleyen bir 
    insan bu cümleye benzer nasıl cümleler demiş olabilir bunlardan 5 adet uzunluğu orijinal cümleyle yakın olan cümleleri
    sadece ve sadece aşağıdaki python liste formatındaki gibi yaz. Başka hiçbir şey yazma:
    Çıktı formatı: ["<cumle1>", "<cumle2>", "<cumle3>", "<cumle4>", "<cumle5>"]
    
    \n\n
    Cümle: """ + sentence)

    try:
        pattern = re.compile(r'\[.*?\]', re.DOTALL)
        # print(response.text)
        match = pattern.search(response.text)
        if match:
            response_json = ast.literal_eval(match.group(0))
            return response_json
        
        else:
            mistake_sentences.append(sentence)
            print("It could not be augmented.")
            return []
            
    except Exception as err:
        mistake_sentences.append(sentence)
        print("It could not be augmented. Err:", err)
        return []

    

## Running Example For SM

In [7]:
selectivemutism_augmented_texts = []

In [8]:
anxiety_total_data = anxiety_total_data.dropna().reset_index(drop=True)

In [12]:
selectivemutism_texts =  anxiety_total_data['Translated_Text'].values

for text in tqdm(selectivemutism_texts):
    if text not in selectivemutism_augmented_texts:  
        selectivemutism_augmented_texts.append(text)
        
        augmented_text_list = submission_prediction(sentence=text)
    
        selectivemutism_augmented_texts.extend(augmented_text_list)
        time.sleep(4.1)

100%|██████████████████████████████████████████████████████████████████████████████| 1500/1500 [12:21<00:00,  2.02it/s]


In [15]:
len(selectivemutism_augmented_texts)

8930

In [14]:
import pickle

with open('../Data/pmdd_1_augmented_data.pkl', 'wb') as file:
    pickle.dump(selectivemutism_augmented_texts, file)